In [ ]:
import pandas as pd 

train = pd.read_csv('./test.csv', sep='\t')
test = pd.read_csv('./train.csv', sep='\t')

df = positive.append(negative)
train.head()


,label,review
0,others,Don't worry I'm girl hmm how do I know if you...
1,angry,When did I? saw many times i think -_- No. I n...
2,others,By by Google Chrome Where you live
3,angry,U r ridiculous I might be ridiculous but I am ...
4,others,Just for time pass wt do u do 4 a living then ...


In [ ]:
! pip install pymorphy2

     |████████████████████████████████| 61kB 7.6MB/s 
     |████████████████████████████████| 8.2MB 17.5MB/s 


Возможно стоит заменить процесс лемматизации на Экфразис

In [ ]:
import pymorphy2
from functools import lru_cache
from multiprocessing import Pool
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import re

# pymorphy2 - библиотека методов для морфологического анализа (в том числе лемматизации) русскоязычного текста
m = pymorphy2.MorphAnalyzer()

# убираем все небуквенные символы
regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return []

In [ ]:
#@lru_cache(maxsize=128)
# если вы работаете не колабе, можно заменить pymorphy на mystem и раскомментирвать первую строку про lru_cache
def lemmatize(text, pymorphy=m):
    try:
        return " ".join([pymorphy.parse(w)[0].normal_form for w in text])
    except:
        return " "

In [ ]:
def clean_text(text):
    return lemmatize(words_only(text))

In [ ]:
# распараллелим процесс на 8 копий, чтобы ускорить, 
# и к каждому объекту датасета ( = твиту) применим написанную выше функцию препроцессинга

with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['review']), total=len(df)))
    
df['lemmas'] = lemmas
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


,label,review,lemmas
0,others,Don't worry I'm girl hmm how do I know if you...,don t worry i m girl hmm how do i know if you ...
1,angry,When did I? saw many times i think -_- No. I n...,when did i saw many times i think _ no i never...
2,others,By by Google Chrome Where you live,by by google chrome where you live
3,angry,U r ridiculous I might be ridiculous but I am ...,u r ridiculous i might be ridiculous but i am ...
4,others,Just for time pass wt do u do 4 a living then ...,just for time pass wt do u do a living then maybe


In [ ]:
# переводим данные из датафрейма в списки

X = df.lemmas.tolist()
y = df.label.tolist()

X, y = np.array(X), np.array(y)

# разбиваем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 60597
total test examples 20200


In [ ]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')
    

with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

In [ ]:
!pip install fasttext
import fasttext
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

     |████████████████████████████████| 71kB 7.3MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3097328 sha256=a83c0a598280fc164aa4b079886b669d2e2764846accc1c8e7ed46533e544ff5
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
print('Precision:', result[1])
print('Recall:', result[2])
print('Number of examples:', result[0])

Precision: 0.8449009900990099
Recall: 0.8449009900990099
Number of examples: 20200


In [ ]:
classifier.save_model("/content/drive/MyDrive/всякое/Модельw2v.bin")

In [ ]:
classifier.predict("stress where is that come from	few media outlets in America. Next 24 hours	i am enjoying")

(('__label__happy',), array([0.84545875]))